# Analyse the GA results

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

%matplotlib widget

In [2]:
import os
import numpy as np
import pandas as pd
import logging
import pickle
import jupyter_black
from tqdm import tqdm

from src.data_connectors import write_solution_files
from src.data_connectors import read_input_files

jupyter_black.load()

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

## Read the results

In [3]:
with open("../data/solutions/all_ga_MMTSP_SAC.pkl", "rb") as f:
    loaded_results = pickle.load(f)

In [4]:
len(loaded_results)

120

In [5]:
# First instance
loaded_results[0][-1]

217

In [6]:
# Last instance
loaded_results[-1][-1]

336

## Process results

list[all_surviving_chromosomes, all_surviving_makespans, best_makespan, duration_until_best_makespan]

In [7]:
instances_path = "../data/input/HRTInstances/"
instances_list = [x for x in range(217, 221)]

times_of_best_results = []

for best, instance_number in tqdm(zip(loaded_results, instances_list)):
    ins_x = ins_x = read_input_files.read_file(
        os.path.join(instances_path, f"Instance_{instance_number}.txt")
    )
    times_of_best_results.append(write_solution_files.find_times_of_best_solution(ins_x, best))

4it [00:00, 12.57it/s]


In [8]:
times_of_best_results[0]

{'0': {2: (2, 37),
  22: (38, 73),
  3: (74, 79),
  23: (80, 85),
  4: (86, 116),
  24: (117, 147),
  5: (148, 247),
  25: (248, 347),
  6: (348, 384),
  26: (385, 421),
  7: (422, 463),
  27: (464, 505),
  9: (506, 526),
  29: (527, 547),
  10: (548, 646),
  30: (647, 745),
  12: (746, 790),
  32: (791, 835),
  13: (836, 883),
  33: (884, 931),
  14: (932, 969),
  34: (970, 1007),
  15: (1008, 1069),
  35: (1070, 1131),
  17: (1132, 1162),
  37: (1163, 1193),
  18: (1194, 1221),
  38: (1222, 1249),
  20: (1250, 1338),
  40: (1339, 1427)},
 '1': {1: (0, 1), 8: (38, 48), 11: (49, 54), 16: (791, 796), 19: (797, 891)},
 '2': {21: (0, 1), 28: (74, 84), 31: (85, 90), 36: (836, 841), 39: (842, 936)}}

## Optimal Solutions

In [9]:
df_optimal = pd.read_csv("../data/solutions/optimal/OptimalSolutions.csv", sep=";", header=1)

df_optimal.head()

,Instance,Humans,Robots,Robot Eligibility,Optimal solution,Best known solution,Lower bound
0,217,1,2,"0,25",1397.0,1397,1397
1,218,1,2,"0,5",865.0,865,865
2,219,1,2,1,529.0,529,529
3,220,1,2,"0,25",3316.0,3316,3316
4,221,1,2,"0,5",2476.0,2476,2476


In [10]:
def get_gap(df_optimal: pd.DataFrame, instance_number: int, best: tuple) -> float:
    lower_bound = df_optimal[df_optimal.Instance == instance_number]["Lower bound"].values[0]
    ins_result = best[2]
    gap = (ins_result - lower_bound) / lower_bound * 100
    logging.debug(f"Lower Bound: {lower_bound}, Result {ins_result}")
    logging.debug(f"Instance {instance_number} has Gap % = {gap:.2f} %")
    return gap

In [11]:
instances_list = [x for x in range(217, 336)]

pct_deviation_from_best = []

for best in tqdm(loaded_results):
    instance_number = best[-1]
    optimal = df_optimal[df_optimal.Instance == instance_number]["Optimal solution"].values[0]
    best_know = df_optimal[df_optimal.Instance == instance_number]["Best known solution"].values[0]
    logging.debug(f"Optimal: {optimal}, Best Known: {best_know}")
    gap = get_gap(df_optimal, instance_number, best)
    logging.info(f"Instance {instance_number} has Gap % = {gap:.2f} %")

  0%|          | 0/120 [00:00<?, ?it/s]INFO:root:Instance 217 has Gap % = 2.15 %
INFO:root:Instance 218 has Gap % = 3.93 %
INFO:root:Instance 219 has Gap % = 19.47 %
INFO:root:Instance 220 has Gap % = 2.26 %
INFO:root:Instance 221 has Gap % = 5.78 %
INFO:root:Instance 222 has Gap % = 12.07 %
INFO:root:Instance 223 has Gap % = 1.73 %
INFO:root:Instance 224 has Gap % = 1.53 %
INFO:root:Instance 225 has Gap % = 16.05 %
INFO:root:Instance 226 has Gap % = 2.32 %
INFO:root:Instance 227 has Gap % = 8.24 %
INFO:root:Instance 228 has Gap % = 21.79 %
INFO:root:Instance 229 has Gap % = 10.09 %
INFO:root:Instance 230 has Gap % = 7.85 %
INFO:root:Instance 231 has Gap % = 142.37 %
INFO:root:Instance 232 has Gap % = 13.64 %
INFO:root:Instance 233 has Gap % = 9.56 %
INFO:root:Instance 234 has Gap % = 53.10 %
INFO:root:Instance 235 has Gap % = 7.98 %
INFO:root:Instance 236 has Gap % = 4.42 %
INFO:root:Instance 237 has Gap % = 14.84 %
INFO:root:Instance 238 has Gap % = 6.70 %
INFO:root:Instance 239 has 

## Group by type

In [12]:
df_instance_groups = (
    df_optimal.groupby(["Humans", "Robots", "Robot Eligibility"]).Instance.apply(list).reset_index()
)
df_instance_groups


all_gaps = []
for idx, row_group in tqdm(df_instance_groups.iterrows()):
    gap_group = []
    for best in tqdm(loaded_results):
        instance_number = best[-1]
        gap_x = get_gap(df_optimal, instance_number, best)
        gap_group.append(gap_x)
    all_gaps.append(gap_group)

df_instance_groups["Gaps"] = all_gaps
df_instance_groups["Gaps%"] = df_instance_groups["Gaps"].apply(lambda x: sum(x) / len(x))

100%|██████████| 120/120 [00:00<00:00, 9098.60it/s]
12it [00:00, 67.00it/s]


In [13]:
df_instance_groups

,Humans,Robots,Robot Eligibility,Instance,Gaps,Gaps%
0,1,2,"0,25","[217, 220, 229, 232, 241, 244, 253, 256, 265, ...","[2.1474588403722263, 3.9306358381502893, 19.47...",42.832898
1,1,2,"0,5","[218, 221, 230, 233, 242, 245, 254, 257, 266, ...","[2.1474588403722263, 3.9306358381502893, 19.47...",42.832898
2,1,2,1,"[219, 222, 231, 234, 243, 246, 255, 258, 267, ...","[2.1474588403722263, 3.9306358381502893, 19.47...",42.832898
3,1,3,"0,25","[223, 226, 235, 238, 247, 250, 259, 262, 271, ...","[2.1474588403722263, 3.9306358381502893, 19.47...",42.832898
4,1,3,"0,5","[224, 227, 236, 239, 248, 251, 260, 263, 272, ...","[2.1474588403722263, 3.9306358381502893, 19.47...",42.832898
5,1,3,1,"[225, 228, 237, 240, 249, 252, 261, 264, 273, ...","[2.1474588403722263, 3.9306358381502893, 19.47...",42.832898
6,2,2,"0,25","[277, 280, 289, 292, 301, 304, 313, 316, 325, ...","[2.1474588403722263, 3.9306358381502893, 19.47...",42.832898
7,2,2,"0,5","[278, 281, 290, 293, 302, 305, 314, 317, 326, ...","[2.1474588403722263, 3.9306358381502893, 19.47...",42.832898
8,2,2,1,"[279, 282, 291, 294, 303, 306, 315, 318, 327, ...","[2.1474588403722263, 3.9306358381502893, 19.47...",42.832898
9,2,3,"0,25","[283, 286, 295, 298, 307, 310, 319, 322, 331, ...","[2.1474588403722263, 3.9306358381502893, 19.47...",42.832898
